In [0]:
movies = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("dbfs:/FileStore/shared_uploads/italomarcelo@outlook.com/netflix_titles-2.csv")

In [0]:
movies.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [0]:
movies.show(3)

+-------+-------+-----+-----------------+--------------------+---------+-----------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|title|         director|                cast|  country|       date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+-----+-----------------+--------------------+---------+-----------------+------------+------+---------+--------------------+--------------------+
|     s1|TV Show|   3%|             null|João Miguel, Bian...|   Brazil|  August 14, 2020|        2020| TV-MA|4 Seasons|International TV ...|In a future where...|
|     s2|  Movie| 7:19|Jorge Michel Grau|Demián Bichir, Hé...|   Mexico|December 23, 2016|        2016| TV-MA|   93 min|Dramas, Internati...|After a devastati...|
|     s3|  Movie|23:59|     Gilbert Chan|Tedd Chan, Stella...|Singapore|December 20, 2018|        2011|     R|   78 min|Horror Movies, In...|When an army recr...|
+-------+-------+-----

In [0]:
df = movies.select('title', 'release_year', 'country', 'rating')
df.show(4)

+-----+------------+-------------+------+
|title|release_year|      country|rating|
+-----+------------+-------------+------+
|   3%|        2020|       Brazil| TV-MA|
| 7:19|        2016|       Mexico| TV-MA|
|23:59|        2011|    Singapore|     R|
|    9|        2009|United States| PG-13|
+-----+------------+-------------+------+
only showing top 4 rows



In [0]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- country: string (nullable = true)
 |-- rating: string (nullable = true)



In [0]:
df2 = df.withColumn('year', df['release_year'].cast('int')).drop('release_year')
df2.printSchema()

root
 |-- title: string (nullable = true)
 |-- country: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- year: integer (nullable = true)



In [0]:
df2.filter('year > 2015').show(5)

+-----+-------+------+----+
|title|country|rating|year|
+-----+-------+------+----+
|   3%| Brazil| TV-MA|2020|
| 7:19| Mexico| TV-MA|2016|
|   46| Turkey| TV-MA|2016|
|  122|  Egypt| TV-MA|2019|
|  706|  India| TV-14|2019|
+-----+-------+------+----+
only showing top 5 rows



In [0]:
# min e max
from pyspark.sql.functions import max, min

In [0]:
df2.select(max('year')).show(3)

+---------+
|max(year)|
+---------+
|     2021|
+---------+



In [0]:
df2.filter('year == 2021 and country is not NULL').show(10)

+--------------------+--------------------+------+----+
|               title|             country|rating|year|
+--------------------+--------------------+------+----+
|     Carmen Sandiego|       United States| TV-Y7|2021|
|            Charming|Canada, United St...| TV-Y7|2021|
|           Cobra Kai|       United States| TV-14|2021|
|Crack: Cocaine, C...|       United States| TV-MA|2021|
|      Disenchantment|       United States| TV-14|2021|
| Dream Home Makeover|       United States|  TV-G|2021|
|Headspace Guide t...|       United States|  TV-G|2021|
|               Hilda|United Kingdom, C...| TV-Y7|2021|
|History of Swear ...|       United States| TV-MA|2021|
|Inside the World’...|      United Kingdom| TV-MA|2021|
+--------------------+--------------------+------+----+
only showing top 10 rows



In [0]:
# CONTAR PAISES
from pyspark.sql.functions import count, col, asc, desc

In [0]:
a = df2.filter('country == "India" ').count()
a

Out[163]: 923

In [0]:
b = df2.groupBy('country').count()
b.show(3)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|Peru, United Stat...|    1|
|      India, Germany|    2|
|Japan, Canada, Un...|    1|
+--------------------+-----+
only showing top 3 rows



In [0]:
b.orderBy(col('count').desc()).show(truncate=False)

+-----------------------------+-----+
|country                      |count|
+-----------------------------+-----+
|United States                |2543 |
|India                        |923  |
|null                         |509  |
|United Kingdom               |397  |
|Japan                        |226  |
|South Korea                  |183  |
|Canada                       |177  |
|Spain                        |134  |
|France                       |114  |
|Egypt                        |101  |
|Mexico                       |100  |
|Turkey                       |100  |
|Australia                    |83   |
|Taiwan                       |78   |
|Brazil                       |72   |
|Philippines                  |71   |
|Indonesia                    |70   |
|Nigeria                      |70   |
|United Kingdom, United States|64   |
|Germany                      |61   |
+-----------------------------+-----+
only showing top 20 rows



In [0]:
# in e contains
a = df.filter('country == "United States"').count()
b = df.filter(df['country'].isin('United States')).count()
c = df.filter(df['country'].contains('United States')).count()
# negando
d = df.filter(~df['country'].isin('United States')).count()
e = df.filter(~df['country'].contains('United States')).count()

a, b, c, d, e

Out[166]: (2543, 2543, 3285, 4737, 3995)

In [0]:
# usando sql
df.createOrReplaceTempView('movies_temp')
spark.sql('select country, count(*) as qtd from movies_temp group by country order by qtd desc').show(truncate=False)

+-----------------------------+----+
|country                      |qtd |
+-----------------------------+----+
|United States                |2543|
|India                        |923 |
|null                         |509 |
|United Kingdom               |397 |
|Japan                        |226 |
|South Korea                  |183 |
|Canada                       |177 |
|Spain                        |134 |
|France                       |114 |
|Egypt                        |101 |
|Mexico                       |100 |
|Turkey                       |100 |
|Australia                    |83  |
|Taiwan                       |78  |
|Brazil                       |72  |
|Philippines                  |71  |
|Indonesia                    |70  |
|Nigeria                      |70  |
|United Kingdom, United States|64  |
|Germany                      |61  |
+-----------------------------+----+
only showing top 20 rows



In [0]:
dfr = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/italomarcelo@outlook.com/netflix_rating.csv")
dfr.printSchema()

root
 |-- rating: string (nullable = true)
 |-- ratingLevel: string (nullable = true)



In [0]:
mj = movies.join(dfr, movies.rating == dfr.rating, 'inner')\
    .select(movies.title, movies.country, movies.release_year, dfr.ratingLevel) \
    .filter(df['country'].contains('Brazil')) \
    .filter(df['country'].contains('France')) \
    .sort(movies.release_year.desc())
mj.show(200, truncate=True)

+--------------------+--------------------+------------+--------------------+
|               title|             country|release_year|         ratingLevel|
+--------------------+--------------------+------------+--------------------+
|        Wasp Network|France, Brazil, S...|        2020|For mature audien...|
|     Shine Your Eyes|      Brazil, France|        2020|For mature audien...|
|    A Sort of Family|Argentina, Brazil...|        2017|Parents strongly ...|
|Gabriel and the M...|      Brazil, France|        2017|For mature audien...|
|  Olmo & the Seagull|Denmark, Brazil, ...|        2014|For mature audien...|
|         City of God|Brazil, France, G...|        2002|strong violence |...|
+--------------------+--------------------+------------+--------------------+



In [0]:
mj.write.csv('netflix_brfr.csv')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2594027647989406> in <module>
----> 1 mj.write.csv('netflix_brfr.csv')

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
    953                        charToEscapeQuoteEscaping=charToEscapeQuoteEscaping,
    954                        encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
--> 955         self._jwrite.csv(path)
    956 
    957     def orc(self, path, mode=None, partitionBy=None, compression=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-